In [1]:
import team_individual_stat
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from nba_api.stats.endpoints import playergamelog

In [2]:
#!pip install nba_api
#!pip install seaborn

## RETRIEVE DATA FOR ML PART

In [9]:
def fetch_last_n_games(player_id, season, n, season_type):
    """
    Fetches the last N completed games of a player from NBA API for a given season.
    
    Parameters:
        player_id (int): The unique ID of the player.
        season (str): NBA season in 'YYYY-YY' format (e.g., '2024-25').
        n (int): Number of last games to retrieve.
        season_type (str): Type of season ('Regular Season' or 'Playoffs').
        
    Returns:
        pd.DataFrame: A DataFrame containing the last N completed games.
    """
    try:
        # Fetch the game logs
        time.sleep(1)  # Prevent hitting the rate limit
        logs = playergamelog.PlayerGameLog(player_id=player_id, season=season, season_type_all_star=season_type)
        logs_df = logs.get_data_frames()[0]
        
        # Ensure GAME_DATE is a datetime object and sort by date
        logs_df['GAME_DATE'] = pd.to_datetime(logs_df['GAME_DATE'])
        logs_df = logs_df.sort_values(by='GAME_DATE', ascending=False)
        
        # Filter out games that have not been completed (those missing 'MIN')
        completed_games = logs_df[logs_df['MIN'].notna()]
        
        # Return only the last n completed games
        last_n_games = completed_games.head(n)
        
        return last_n_games[['GAME_DATE', 'PTS', 'AST']]
    except Exception as e:
        print(f"Error retrieving last {n} completed games for player {player_id}: {e}")
        return pd.DataFrame()

In [13]:
last_reg_df = fetch_last_n_games(
    player_id=2544, # Lebron James
    season='2023-24',
    n=100,
    season_type='Regular Season'
)


In [14]:
last_reg_df.head()

,GAME_DATE,PTS,AST
0,2024-04-14,28,17
1,2024-04-12,37,5
2,2024-04-09,33,11
3,2024-04-06,24,12
4,2024-04-03,25,9


In [16]:
last_playoff_df = fetch_last_n_games(
    player_id=2544,
    season='2023-24',
    n=100,  # lớn hơn số trận playoff có thể xảy ra
    season_type='Playoffs'
)


In [18]:
first_playoff_game = last_playoff_df.sort_values('GAME_DATE').iloc[0]


In [20]:
y_true = first_playoff_game[['PTS', 'AST']]
print("first last playoff", y_true.to_dict())


first last playoff {'PTS': 27, 'AST': 8}


# TIME SERIES

In [280]:
#!pip install tensorflow


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 25.0.1
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip
